In [1]:
!apt install -q -y libspatialindex-dev libproj-dev proj-data proj-bin libgeos-dev
!npm install -g mapshaper

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/usr/local/bin/mapshaper -> /usr/local/lib/node_modules/mapshaper/bin/mapshaperycle http-basic@6.0.0~p
/usr/local/bin/mapshaper-xl -> /usr/local/lib/node_modules/mapshaper/bin/mapshaper-xl
/usr/local/bin/mapshaper-gui -> /usr/local/lib/node_modules/mapshaper/bin/mapshaper-gui
+ mapshaper@0.5.115
added 67 packages from 79 contributors in 10.963s

┌──────────────────────────────────────────────────────────┐
│                 npm update check failed                  │
│           Try running with sudo or get access            │
│           to the local update config store via           │
│ sudo chown -R $USER:$(id -gn $USER) /Users/sofia/.config │
└──────────────────────────────────────────────────────────┘


In [ ]:
import os
import sys
from pathlib import Path
import subprocess
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from IPython.display import display, Markdown

In [12]:
WORKING_DIR.parents[1]

PosixPath('/Users/sofia/Documents')

In [13]:
WORKING_DIR = Path(os.getcwd())
DATA = Path(f'{WORKING_DIR.parents[1]}/HE_Data/NRC/EEZ_GADM_MERGED_ORIGINAL_GEOM')
dataPath  = list(DATA.glob('*.shp'))[0]

In [14]:
data_gpd = gpd.read_file(dataPath)

In [15]:
data_gpd.head()

,GID_0,Shape_Leng,Shape_Area,geometry
0,ABW,8.600131,2.504937,"POLYGON ((-68.87017 15.24185, -68.88133 15.200..."
1,AGO,49.857334,41.178688,"MULTIPOLYGON (((13.42086 -5.87340, 13.39728 -5..."
2,AIA,12.561376,7.784427,"POLYGON ((-60.73667 20.92203, -60.76437 20.877..."
3,ALB,10.366878,1.300663,"POLYGON ((20.00835 39.69182, 20.00810 39.69187..."
4,ARE,39.797245,5.174869,"MULTIPOLYGON (((56.84117 25.63989, 56.88185 25..."


In [16]:
def interior_to_polygon(geom: Polygon):
  """
  Takes a Shapely geometry and returns interiors as Shapely geometry.
  """
  types = [geom.geom_type] if type(geom.geom_type) == str else geom.geom_type.tolist()
  # check geometry is a polygon 
  if types and 'Polygon' in types:
    # check it contains a ring
    ring = geom.is_ring
    if ring.any() and geom.interiors.to_list()[0]:
      # extract shapely LinearRing(s)
      rings_list = [Polygon(g) for g in geom.interiors.to_list()[0]]
      out = MultiPolygon(rings_list) if len(rings_list) > 1 else rings_list[0]
    else:
      print("No interior ring")
      out = "NA"
  else:
    print("Requires a Polygon")
    out = "NA"     
  return out

def exterior_to_polygon(geom):
  """
  Takes a Shapely geometry and returns exterior ring as Shapely geometry.
  """
  types = geom.geom_type.tolist() or []
  # check geometry is a polygon 
  if types and 'Polygon' in types:
    # check it contains a ring
    ring = geom.is_ring
    if ring.any() and geom.exterior.tolist()[0]:
      # extract shapely LinearRing
      out = Polygon(geom.exterior.tolist()[0])
    else:
      print("No exterior ring")
      out = "NA"
  else:
    print("Requires a Polygon")
    out = "NA"     
  return out

In [17]:
def execute_command(command):
  """
  Executes a command in the shell and returns the output.
  """
  try:
    out = subprocess.check_output(command, shell=True)
  except subprocess.CalledProcessError as e:
    out = e.output
  return out

In [18]:
o = f"{dataPath}_clean.shp"
execute_command(f'mapshaper-xl 16 -i snap {dataPath} \
        -clean gap-fill-area=30km2 \
        -dissolve GID_0 \
        -o format=shapefile {o} force')

Allocating 16 GB of heap memory
[i] Snapped 103981 points
[clean] Removed 3,877,160 / 3,877,425 slivers using 30+ sqkm variable threshold
[clean] Retained 455 of 457 features
[dissolve] Dissolved 455 features into 253 features
[o] Wrote /Users/sofia/Documents/HE_Data/NRC/EEZ_GADM_MERGED_ORIGINAL_GEOM/EEZ_GADM_MERGED_ORIGINAL_GEOM.shp_clean.shp
[o] Wrote /Users/sofia/Documents/HE_Data/NRC/EEZ_GADM_MERGED_ORIGINAL_GEOM/EEZ_GADM_MERGED_ORIGINAL_GEOM.shp_clean.shx
[o] Wrote /Users/sofia/Documents/HE_Data/NRC/EEZ_GADM_MERGED_ORIGINAL_GEOM/EEZ_GADM_MERGED_ORIGINAL_GEOM.shp_clean.dbf
[o] Wrote /Users/sofia/Documents/HE_Data/NRC/EEZ_GADM_MERGED_ORIGINAL_GEOM/EEZ_GADM_MERGED_ORIGINAL_GEOM.shp_clean.prj


b''